<a href="https://colab.research.google.com/github/richardachen/Fireproof/blob/main/Downloading%20GeoTIFF/Temperature_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages

import ee
from ee import batch
from pandas.tseries.offsets import DateOffset
import pandas as pd

In [ ]:
# initialization
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QxW7Evr2NWB3yWkehTlQ80yafVtPKKFDvHpTboMSqd8&tc=3-34iNyh0M7YWmsV-iwvIUwjseFpBIsflgPnpT84dcw&cc=2-7Mfx9EtZIpa9BC9jt8BI3cosY9GmF5GExv8R2d2OQ

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjV-flG6Ef_9iAhqOr0tupXvh3PyTgDA8aanfbl2yzaiCbrtDQBRqY

Successfully saved authorization token.


In [ ]:
# Woolsey Fire dates
start_date = '2018-11-05'
end_date = '2018-11-24'

# get climate data
lst = ee.ImageCollection('MODIS/006/MOD11A1')
lst_collection = ee.ImageCollection(lst.select('LST_Day_1km')).filterDate(start_date, end_date)

In [ ]:
# information to save files later on
folder_name = 'temperature_maps'
scale = 250

# Ooo, cool note: "While we need to have an eye on the frequency, 
# GEE takes care of resolution and projection by resampling and 
# reprojecting all data we are going to work with to a common projection"
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

# Woolsey Fire locations
la_lon = -117.9267511
la_lat = 34.2817777
la_poi = ee.Geometry.Point(la_lon, la_lat)
la = la_poi.buffer(1e5) #1e5 meters

In [ ]:
# collection = ee.ImageCollection
crs = ee.Image(lst_collection.first()).projection()

def resample(image):
  """
  Resample to new resolution
  """
  image = image.resample('bilinear').reproject(crs=crs, scale=scale)
  return image
if scale != 250:
  lst_collection = lst_collection.map(resample)

In [ ]:
n = lst_collection.size().getInfo()
colList = lst_collection.toList(n)

test_image = ee.Image(colList.get(2))
info_stuff = test_image.getInfo()
print(info_stuff['properties']['system:index'])

2018_11_07


In [ ]:
# export GeoTIFF 
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api#save_a_geotiff_file_in_your_google_drive

for i in range(n):
  image = ee.Image(colList.get(i))
  id = image.id().getInfo() or 'image_'+i.toString()
  info_stuff = image.getInfo()
  date = info_stuff['properties']['system:index']

  out = ee.batch.Export.image.toDrive(image=image,
                                      description='land_temp_in_greater_los_angeles_area',
                                      folder=folder_name,
                                      scale=250,
                                      region=la,
                                      fileNamePrefix='my_export_la_' + date,
                                      crs='EPSG:4326',
                                      fileFormat='GeoTIFF',
                                      maxPixels=1e11)
  batch.Task.start(out)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
